In [1]:
import pandas as pd
import numpy as np

Notes and explanations:
The collegeTransfers function outputs 5 dataframes in respective order:

    - Colleges_allTime: gives the "all time" tallies for the colleges that were transferred to; tallies for pell, fg, gender, race and graduation
    
    - Colleges_lastCompleteCohort: gives the tallies for the colleges that were transferred to by the most recent cohort to complete their 3yr window; tallies for pell, fg, gender, race and graduation
    
    - Colleges_last5CompleteCohorts: gives the tallies for the colleges that were transferred to by the most recent 5 cohorts to complete their 3 yr windows; tallies for pell, fg, gender, race and graduation
    
    - allTransfers: the non-duplicated transfer records since 2013; tech_ids are removed
    
    - publicVSprivate_type_state: combines Colleges_allTime and Colleges_lastCompleteCohort to obtain and instead of tallying for the colleges, tallies the race, collegeType, and PubVPriv against Pell, firstgen and gender

The collegeTransfers function uses data from the below tables:

    - ISRS.ST_COU: the last semester a student attended RCC

    - APPSODS.V_STUDENT_PERSISTENCE_DTL2: each student's final record of retention, transfer, and graduation, pell eligibility, and first generation status

    - APPSODS.V_IPEDS_ETHNIC_RACE: what a students ethnicity/race is

    - ODS.ST_CLEARINGHOUSE: all colleges a student attended and the semester they begain there; IAT information about those colleges (type, pvp, and state)


The TransferTally functions output 4 dataframes in respective order:

    - df1: the overall totals for all non-high school student transfers by FY/term

    - df2: the race, transfer count/rate, and group size by FY/term

    - df3: the firstgen status, transfer count/rate, and group size by FY/term

    - df4: each ind student's tech_id, transfer status, graduation status, cohort year/term, admission status, first gen status, and race


TransferTally functions use data from the below tables:

    - ISRS.ST_COU: the last semester a student attended RCC

    - ISRS.ST_TERM_MGMT: the admission status of the a student during their final RCC semester

    - APPSODS.V_STUDENT_PERSISTENCE_DTL2: each student's final record of retention, transfer, and graduation.

    - APPSODS.V_IPEDS_ETHNIC_RACE: what a students ethnicity/race is
    
    - APPSODS.V_FIRST_GENERATION: whether or not a student is first generation in their family to go to college

Est. Run Times:
        fallTransferTally - <1s  ||
        springTransferTally - <1s  ||
        collegeTransfers - 12s

*INPUTS*

collegeTransfers:

    - df1 = persDTL2_10

    - df2 = last

    - df3 = race_10

    - df4 = transferRecords_10

fallTransferTally:

    - df1 = persDTL2_10 filtered out current FY and Spring starting cohorts --> persDTL_fall 

    - df2 = last

    - df3 = admstat_10

    - df4 = race_10

collegeTransfers:

    - df1 = persDTL2_10

    - df2 = last

    - df3 = race_10

    - df4 = transferRecords_10

springTransferTally:

    - df1 = persDTL2_10 filtered out current FY and Fall starting cohorts --> persDTL_spring

    - df2 = last

    - df3 = admstat_10

    - df4 = race_10

In [2]:
##retrieve data
persDTL2_10 = pd.read_csv('persDTL2_10.csv') #df1   now includes first gen, pell, and gender
courseRecords_10 = pd.read_csv('courseRecords_10.csv')
transferRecords_10 = pd.read_csv('transferRecords_10.csv')
race_10 = pd.read_csv('race_10.csv') #df4
admstat_10 = pd.read_csv('admstat_10.csv') #df3
last = pd.read_csv('lastsemester_10yrs.csv') #df2

In [3]:
persDTL = persDTL2_10.loc[persDTL2_10['FY']!=2024,:]
persDTL_fall = persDTL.loc[persDTL['Semester']==3,:]
persDTL_spring = persDTL.loc[persDTL['Semester']==5,:]

In [4]:
def collegeTransfers(df1, df2, df3, df4):
    #make all your copies
    persDTL_2 = df1.copy()
    last2 = df2.copy()
    race2 = df3.copy()
    tr10 = df4.copy()
    #make your output df
    colleges = pd.DataFrame(columns=['College','Total','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    colleges_currentFY = pd.DataFrame(columns=['College','FY2024','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    colleges_lastCompleteCohort = pd.DataFrame(columns=['College','FY2020','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    colleges_last5CompleteCohorts = pd.DataFrame(columns=['College','FY2016','FY2017','FY2018','FY2019','FY2020','Pell','FirstGen','Male','Female','WHITE','HISP','ASIAN','BLACK','AMER_IND_AK','NAT_HAW_PI','TWO_PLUS','NRA','UNKNOWN','Grad','Grad_at','College_type','State','Pub_Priv'])
    #slice to get only what you need
    persDTL_2['transferred_at'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED'] + persDTL_2['FOURTH_SPRING_TRANSFERRED'] + persDTL_2['FIFTH_FALL_TRANSFERRED'] + persDTL_2['FIFTH_SPRING_TRANSFERRED'] + persDTL_2['SIXTH_FALL_TRANSFERRED'] + persDTL_2['SIXTH_SPRING_TRANSFERRED'] + persDTL_2['SEVENTH_FALL_TRANSFERRED'] + persDTL_2['SEVENTH_SPRING_TRANSFERRED'] + persDTL_2['EIGTH_FALL_TRANSFERRED'] + persDTL_2['EIGTH_SPRING_TRANSFERRED']
    persDTL2_slice = persDTL_2[['TECH_ID','FY','Semester','FIRST_GENERATION_STATUS_MN','PELL_ELIGIBLE','GENDER','transferred_at','COHORT_ENTERING_YEAR_TERM_CODE']]
    slice1 = persDTL2_slice.loc[persDTL2_slice['transferred_at'] > 0,:]
    last_slice = last2.loc[last['YRTR']>20125,:]
    tr10['TRANSFER_YRTR'].fillna(tr10['GRADUATION_YRTR'], inplace=True)
    tr10_2 = tr10.drop(columns=['GRADUATION_YRTR'])
    ##merge all the slices together before working with the transfer records
    glued1 = pd.merge(slice1, last_slice, how='left',on='TECH_ID')
    glued2 = pd.merge(glued1, race2, how='left',on='TECH_ID')
    #merge with transfer records
    mass = pd.merge(glued2, tr10_2, how='inner',on='TECH_ID')
    mass2 = mass.loc[mass['TRANSFER_YRTR'] > mass['COHORT_ENTERING_YEAR_TERM_CODE'],['TECH_ID','COLLEGE_NAME','COLLEGE_STATE','COLLEGE_TYPE','TRANSFER_YRTR','COHORT_ENTERING_YEAR_TERM_CODE','GRADUATE','PUBLIC_PRIVATE','ENROLLMENT_STATUS']]
    #######################################
    ##not sure if we will use this df yet
    grads = mass2.loc[mass2['GRADUATE']=='Y',:]
    #######################################
    #first college after RCC df
    nonRCC = mass2.loc[(mass2['COLLEGE_NAME'] != 'RIVERLAND COMMUNITY COLLEGE'),:]
    firstNew = nonRCC.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID','COLLEGE_NAME'], keep='first', ignore_index=True)
    firstNew2 = firstNew.sort_values(by=['COLLEGE_TYPE'], ascending=False)
    firstNew3 = firstNew2.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID'], keep='first', ignore_index=True)
    lastNew = nonRCC.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID','COLLEGE_NAME'], keep='last', ignore_index=True)
    lastNew2 = lastNew.sort_values(by=['GRADUATE'])
    lastNew3 = lastNew2.sort_values(by=['TECH_ID','TRANSFER_YRTR']).drop_duplicates(subset=['TECH_ID'], keep='last', ignore_index=True)
    ##df with student info, first college info and last college info
    lastFirstNew = pd.merge(firstNew3, lastNew3, how='left', on='TECH_ID', suffixes=('_FIRST','_LAST'))
    mass3 = pd.merge(glued2, lastFirstNew, on='TECH_ID', how='left')
    mass4 = mass3.drop(columns=['COHORT_ENTERING_YEAR_TERM_CODE_FIRST','COHORT_ENTERING_YEAR_TERM_CODE_LAST'])
    colls = mass4[['COLLEGE_NAME_FIRST','COLLEGE_TYPE_FIRST','COLLEGE_STATE_FIRST','PUBLIC_PRIVATE_FIRST']].drop_duplicates().reset_index()
    FY20 = mass4.loc[mass4['FY']==2020,:]
    last5 = mass4.loc[mass4['FY']>2015,:]
    for i in colls.index:
        col = colls.loc[i,'COLLEGE_NAME_FIRST']
        typ = colls.loc[i,'COLLEGE_TYPE_FIRST']
        state = colls.loc[i,'COLLEGE_STATE_FIRST']
        prp = colls.loc[i,'PUBLIC_PRIVATE_FIRST']
        college_allTime = pd.DataFrame(
            {'College':col,
            'Total':mass4.loc[mass4['COLLEGE_NAME_FIRST']==col,'TECH_ID'].count(),
            'Pell':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['PELL_ELIGIBLE']=='Yes'),'TECH_ID'].count(),
            'FirstGen':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['FIRST_GENERATION_STATUS_MN']=='Yes'),'TECH_ID'].count(),
            'Male':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['GENDER']=='Male'),'TECH_ID'].count(),
            'Female':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['GENDER']=='Female'),'TECH_ID'].count(),
            'WHITE':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='WHITE'),'TECH_ID'].count(),
            'HISP':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='HISP'),'TECH_ID'].count(),
            'ASIAN':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='ASIAN'),'TECH_ID'].count(),
            'BLACK':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='BLACK'),'TECH_ID'].count(),
            'AMER_IND_AK':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='AMER_IND_AK'),'TECH_ID'].count(),
            'NAT_HAW_PI':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='NAT_HAW_PI'),'TECH_ID'].count(),
            'TWO_PLUS':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='TWO_PLUS'),'TECH_ID'].count(),
            'NRA':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='NRA'),'TECH_ID'].count(),
            'UNKNOWN':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['IPEDS']=='UNKNOWN'),'TECH_ID'].count(),
            'Grad':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['COLLEGE_NAME_FIRST']==mass4['COLLEGE_NAME_LAST']) &(mass4['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'Grad_at':mass4.loc[(mass4['COLLEGE_NAME_FIRST']==col) & (mass4['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'College_type':typ,
            'State':state,
            'Pub_Priv':prp},
            index=[1])
        colleges = pd.concat([colleges,college_allTime])
        colleges_lastCompleteCohort2 = pd.DataFrame(
            {'College':col,
            'FY2020':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'Pell':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['PELL_ELIGIBLE']=='Yes'),'TECH_ID'].count(),
            'FirstGen':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['FIRST_GENERATION_STATUS_MN']=='Yes'),'TECH_ID'].count(),
            'Male':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['GENDER']=='Male'),'TECH_ID'].count(),
            'Female':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['GENDER']=='Female'),'TECH_ID'].count(),
            'WHITE':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='WHITE'),'TECH_ID'].count(),
            'HISP':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='HISP'),'TECH_ID'].count(),
            'ASIAN':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='ASIAN'),'TECH_ID'].count(),
            'BLACK':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='BLACK'),'TECH_ID'].count(),
            'AMER_IND_AK':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='AMER_IND_AK'),'TECH_ID'].count(),
            'NAT_HAW_PI':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='NAT_HAW_PI'),'TECH_ID'].count(),
            'TWO_PLUS':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='TWO_PLUS'),'TECH_ID'].count(),
            'NRA':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='NRA'),'TECH_ID'].count(),
            'UNKNOWN':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['IPEDS']=='UNKNOWN'),'TECH_ID'].count(),
            'Grad':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['COLLEGE_NAME_FIRST']==FY20['COLLEGE_NAME_LAST']) &(FY20['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'Grad_at':FY20.loc[(FY20['COLLEGE_NAME_FIRST']==col) & (FY20['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'College_type':typ,
            'State':state,
            'Pub_Priv':prp},
            index=[1])
        colleges_lastCompleteCohort = pd.concat([colleges_lastCompleteCohort,colleges_lastCompleteCohort2])

        colleges_last5CompleteCohorts2 = pd.DataFrame(
            {'College':col,
            'FY2016':last5.loc[(last5['FY']==2016) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2017':last5.loc[(last5['FY']==2017) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2018':last5.loc[(last5['FY']==2018) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2019':last5.loc[(last5['FY']==2019) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'FY2020':last5.loc[(last5['FY']==2020) & (last5['COLLEGE_NAME_FIRST']==col),'TECH_ID'].count(),
            'Pell':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['PELL_ELIGIBLE']=='Yes'),'TECH_ID'].count(),
            'FirstGen':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['FIRST_GENERATION_STATUS_MN']=='Yes'),'TECH_ID'].count(),
            'Male':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['GENDER']=='Male'),'TECH_ID'].count(),
            'Female':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['GENDER']=='Female'),'TECH_ID'].count(),
            'WHITE':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='WHITE'),'TECH_ID'].count(),
            'HISP':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='HISP'),'TECH_ID'].count(),
            'ASIAN':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='ASIAN'),'TECH_ID'].count(),
            'BLACK':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='BLACK'),'TECH_ID'].count(),
            'AMER_IND_AK':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='AMER_IND_AK'),'TECH_ID'].count(),
            'NAT_HAW_PI':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='NAT_HAW_PI'),'TECH_ID'].count(),
            'TWO_PLUS':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='TWO_PLUS'),'TECH_ID'].count(),
            'NRA':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='NRA'),'TECH_ID'].count(),
            'UNKNOWN':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['IPEDS']=='UNKNOWN'),'TECH_ID'].count(),
            'Grad':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['COLLEGE_NAME_FIRST']==last5['COLLEGE_NAME_LAST']) &(last5['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'Grad_at':last5.loc[(last5['COLLEGE_NAME_FIRST']==col) & (last5['GRADUATE_LAST']=='Y'),'TECH_ID'].count(),
            'College_type':typ,
            'State':state,
            'Pub_Priv':prp},
            index=[1])
        colleges_last5CompleteCohorts = pd.concat([colleges_last5CompleteCohorts,colleges_last5CompleteCohorts2])
    df_out1 = colleges.sort_values(by='Total', ascending=False)
    df_out2 = colleges_lastCompleteCohort.sort_values(by='FY2020', ascending=False)
    df_out3 = colleges_last5CompleteCohorts.sort_values(by='FY2016', ascending=False)
    df_out4 = mass4.drop(columns=['TECH_ID'])
    publicVSprivate_type_state = pd.DataFrame(
        {'Demographic':['ALL','ALL','Female','Female','Male','Male','Pell','FirstGen','Pell','FirstGen'],
        'FY':['ALL','2020','ALL','2020','ALL','2020','ALL','ALL','2020','2020'],
        'Public':[sum(colleges.loc[colleges['Pub_Priv']=='P','Total']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','FY2020']),sum(colleges.loc[colleges['Pub_Priv']=='P','Female']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','Female']), sum(colleges.loc[colleges['Pub_Priv']=='P','Male']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','Male']), sum(colleges.loc[colleges['Pub_Priv']=='P','Pell']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','FirstGen']), sum(colleges.loc[colleges['Pub_Priv']=='P','Pell']),sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='P','FirstGen'])],
        'Private':[sum(colleges.loc[colleges['Pub_Priv']=='V','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','FY2020']),sum(colleges.loc[colleges['Pub_Priv']=='V','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','Female']), sum(colleges.loc[colleges['Pub_Priv']=='V','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','Male']), sum(colleges.loc[colleges['Pub_Priv']=='V','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','FirstGen']), sum(colleges.loc[colleges['Pub_Priv']=='V','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['Pub_Priv']=='V','FirstGen'])],
        '_2yr':[sum(colleges.loc[colleges['College_type']=='2','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','FY2020']), sum(colleges.loc[colleges['College_type']=='2','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','Female']), sum(colleges.loc[colleges['College_type']=='2','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','Male']), sum(colleges.loc[colleges['College_type']=='2','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','FirstGen']), sum(colleges.loc[colleges['College_type']=='2','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='2','FirstGen'])],
        '_4yr':[sum(colleges.loc[colleges['College_type']=='4','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','FY2020']), sum(colleges.loc[colleges['College_type']=='4','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','Female']), sum(colleges.loc[colleges['College_type']=='4','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','Male']), sum(colleges.loc[colleges['College_type']=='4','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','FirstGen']), sum(colleges.loc[colleges['College_type']=='4','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['College_type']=='4','FirstGen'])],
        'MN':[sum(colleges.loc[colleges['State']=='MN','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','FY2020']), sum(colleges.loc[colleges['State']=='MN','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','Female']), sum(colleges.loc[colleges['State']=='MN','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','Male']), sum(colleges.loc[colleges['State']=='MN','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','FirstGen']), sum(colleges.loc[colleges['State']=='MN','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='MN','FirstGen'])],
        'WI':[sum(colleges.loc[colleges['State']=='WI','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','FY2020']), sum(colleges.loc[colleges['State']=='WI','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','Female']), sum(colleges.loc[colleges['State']=='WI','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','Male']), sum(colleges.loc[colleges['State']=='WI','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','FirstGen']), sum(colleges.loc[colleges['State']=='WI','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='WI','FirstGen'])],
        'IA':[sum(colleges.loc[colleges['State']=='IA','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','FY2020']), sum(colleges.loc[colleges['State']=='IA','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','Female']), sum(colleges.loc[colleges['State']=='IA','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','Male']), sum(colleges.loc[colleges['State']=='IA','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','FirstGen']), sum(colleges.loc[colleges['State']=='IA','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IA','FirstGen'])],
        'IL':[sum(colleges.loc[colleges['State']=='IL','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','FY2020']), sum(colleges.loc[colleges['State']=='IL','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','Female']), sum(colleges.loc[colleges['State']=='IL','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','Male']), sum(colleges.loc[colleges['State']=='IL','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','FirstGen']), sum(colleges.loc[colleges['State']=='IL','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='IL','FirstGen'])],
        'ND':[sum(colleges.loc[colleges['State']=='ND','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','FY2020']), sum(colleges.loc[colleges['State']=='ND','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','Female']), sum(colleges.loc[colleges['State']=='ND','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','Male']), sum(colleges.loc[colleges['State']=='ND','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','FirstGen']), sum(colleges.loc[colleges['State']=='ND','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='ND','FirstGen'])],
        'SD':[sum(colleges.loc[colleges['State']=='SD','Total']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','FY2020']), sum(colleges.loc[colleges['State']=='SD','Female']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','Female']), sum(colleges.loc[colleges['State']=='SD','Male']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','Male']), sum(colleges.loc[colleges['State']=='SD','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','FirstGen']), sum(colleges.loc[colleges['State']=='SD','Pell']), sum(colleges_lastCompleteCohort.loc[colleges_lastCompleteCohort['State']=='SD','FirstGen'])],
        'Other':[sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Total']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Female']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Male']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Pell']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'FirstGen']), sum(colleges.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'Pell']), sum(colleges_lastCompleteCohort.loc[(colleges['State']!='MN') & (colleges['State']!='WI') & (colleges['State']!='IA') & (colleges['State']!='IL') & (colleges['State']!='ND') & (colleges['State']!='SD'),'FirstGen'])]
        },
        index = [1,2,3,4,5,6,7,8,9,10])

    return df_out1, df_out2, df_out3, df_out4, publicVSprivate_type_state

In [5]:
def fallTransferTally(df1,df2,df3,df4):
       trdf = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','COHORT_SIZE','TRAN_wi3','TRAN_at','TR'])
       trdf_race = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','RACE','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_FG = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','FIRST_GEN_MN','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_pell = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','PELL_ELG','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_gender = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','GENDER','COHORT_SIZE','TRAN_wi3','TR'])
       persDTL_2 = df1.copy()
       #create and fill new columns
       persDTL_2['transferred_wi3_fall'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED']
       persDTL_2['transferred_at'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED'] + persDTL_2['FOURTH_SPRING_TRANSFERRED'] + persDTL_2['FIFTH_FALL_TRANSFERRED'] + persDTL_2['FIFTH_SPRING_TRANSFERRED'] + persDTL_2['SIXTH_FALL_TRANSFERRED'] + persDTL_2['SIXTH_SPRING_TRANSFERRED'] + persDTL_2['SEVENTH_FALL_TRANSFERRED'] + persDTL_2['SEVENTH_SPRING_TRANSFERRED'] + persDTL_2['EIGTH_FALL_TRANSFERRED'] + persDTL_2['EIGTH_SPRING_TRANSFERRED']
       persDTL_2['grad_at'] = persDTL_2['FIRST_FALL_GRADUATED'] + persDTL_2['FIRST_SPRING_GRADUATED'] + persDTL_2['SECOND_FALL_GRADUATED'] + persDTL_2['SECOND_SPRING_GRADUATED'] + persDTL_2['THIRD_FALL_GRADUATED'] + persDTL_2['THIRD_SPRING_GRADUATED'] + persDTL_2['FOURTH_FALL_GRADUATED'] + persDTL_2['FOURTH_SPRING_GRADUATED'] + persDTL_2['FIFTH_FALL_GRADUATED'] + persDTL_2['FIFTH_SPRING_GRADUATED'] + persDTL_2['SIXTH_FALL_GRADUATED'] + persDTL_2['SIXTH_SPRING_GRADUATED'] + persDTL_2['SEVENTH_FALL_GRADUATED'] + persDTL_2['SEVENTH_SPRING_GRADUATED'] + persDTL_2['EIGTH_FALL_GRADUATED'] + persDTL_2['EIGTH_SPRING_GRADUATED']
       persDTL_2['retained'] = persDTL_2['FIRST_FALL_RETAINED'] + persDTL_2['FIRST_SPRING_RETAINED'] + persDTL_2['SECOND_FALL_RETAINED'] + persDTL_2['SECOND_SPRING_RETAINED'] + persDTL_2['THIRD_FALL_RETAINED'] + persDTL_2['THIRD_SPRING_RETAINED'] + persDTL_2['FOURTH_FALL_RETAINED'] + persDTL_2['FOURTH_SPRING_RETAINED'] + persDTL_2['FIFTH_FALL_RETAINED'] + persDTL_2['FIFTH_SPRING_RETAINED'] + persDTL_2['SIXTH_FALL_RETAINED'] + persDTL_2['SIXTH_SPRING_RETAINED'] + persDTL_2['SEVENTH_FALL_RETAINED'] + persDTL_2['SEVENTH_SPRING_RETAINED'] + persDTL_2['EIGTH_FALL_RETAINED'] + persDTL_2['EIGTH_SPRING_RETAINED']
       #get rid of the lengthy semester columns
       persDTL_3 = persDTL_2[['TECH_ID','FIRST_GENERATION_STATUS_MN','FIRST_GENERATION_STATUS_FED','PELL_ELIGIBLE','GENDER','FY','Semester','transferred_wi3_fall','transferred_at','grad_at','retained']]
       #merge df2 through df5
       last_admstat = pd.merge(df2, df3, how='left',on=['TECH_ID','YRTR'])
       last_admstat_race = pd.merge(last_admstat, df4,how='left',on='TECH_ID')
       #merge with the transfer df
       demo_transfers = pd.merge(persDTL_3, last_admstat_race, how='left',on='TECH_ID')
       fys = demo_transfers['FY'].unique()
       for fy in fys:
              df_main = demo_transfers.loc[(demo_transfers['FY']==fy),:]
              ##df_out1 variables
              cohort_size_main, transferred_wi3_main, transferred_at_main = 0,0,0
              ##df_out2 variables
              two_plus,native_a, pacific_i, hispanic, asian, white, black, nra, unknown = 0,0,0,0,0,0,0,0,0
              two_plus_t, native_a_t, pacific_i_t, hispanic_t, asian_t, white_t, black_t, nra_t, unknown_t = 0,0,0,0,0,0,0,0,0
              TR_white,TR_asian,TR_black,TR_hispanic,TR_two_plus,TR_native_a,TR_pacific_i,TR_nra,TR_unknown = 0,0,0,0,0,0,0,0,0
              ##df_out3 variables
              fg_cs, fg_t = 0,0
              ##df_out4 variables
              pell_elg, pell_elg_t = 0,0
              ##df_out5 variables
              male, female, male_t, female_t = 0,0,0,0
              df_main.reset_index()
              for i in df_main.index:
                     cohort_size_main = cohort_size_main + 1
                     at = df_main.loc[i,'transferred_at']
                     wi3 = df_main.loc[i,'transferred_wi3_fall']
                     fg = df_main.loc[i,'FIRST_GENERATION_STATUS_MN']
                     race = df_main.loc[i,'IPEDS']
                     pell = df_main.loc[i,'PELL_ELIGIBLE']
                     gender = df_main.loc[i,'GENDER']
                     if race == 'WHITE':
                            white = white + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   white_t = white_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'HISP':
                            hispanic = hispanic + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   hispanic_t = hispanic_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'BLACK':
                            black = black + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   black_t = black_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'ASIAN':
                            asian = asian + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   asian_t = asian_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'TWO_PLUS':
                            two_plus = two_plus + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   two_plus_t = two_plus_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'AMER_IND_AK':
                            native_a = native_a + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   native_a_t = native_a_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NAT_HAW_PI':
                            pacific_i = pacific_i + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   pacific_i_t = pacific_i_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NRA':
                            nra = nra + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   nra_t = nra_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'UNKNOWN':
                            unknown = unknown + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   unknown_t = unknown_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1       
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     else:
                            continue
              ##Race transfer rate; this doesn't seem necessary, but some of these groupings are 0 size, which will break that function should a 0 end up in the denominator 
              if white > 0:
                     TR_white = white_t/white*100   
              if asian > 0:
                     TR_asian = asian_t/asian*100   
              if black > 0:
                     TR_black = black_t/black*100   
              if hispanic > 0:
                     TR_hispanic = hispanic_t/hispanic*100   
              if two_plus > 0:
                     TR_two_plus = two_plus_t/two_plus*100 
              if native_a > 0:  
                     TR_native_a = native_a_t/native_a*100   
              if pacific_i > 0:
                     TR_pacific_i = pacific_i_t/pacific_i*100   
              if nra > 0:
                     TR_nra = nra_t/nra*100   
              if unknown > 0:
                     TR_unknown = unknown_t/unknown*100  
              #df: gender
              TR_fem = 0
              if female > 0:
                     TR_fem = female_t/female*100
              TR_male = 0
              if male > 0:
                     TR_male = male_t/male*100
              sdf_gender = pd.DataFrame(
                     {'COHORT_FY':[fy,fy],
                     'COHORT_SEMESTER':['FALL','FALL'],
                     'GENDER':['Female','Male'],
                     'COHORT_SIZE':[female, male],
                     'TRAN_wi3':[female_t, male_t],
                     'TR':[TR_fem, TR_male]}, index=[1,2])
              #df: pell
              TR_pell = 0
              if pell_elg > 0:
                     TR_pell = pell_elg_t/pell_elg*100
              sdf_pell = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['FALL'],
                     'PELL_ELG':['Yes'],
                     'COHORT_SIZE':[pell_elg],
                     'TRAN_wi3':[pell_elg_t],
                     'TR':[TR_pell]}, index=[1])
              #df: race
              sdf_race = pd.DataFrame(
                     {'COHORT_FY':[fy,fy,fy,fy,fy,fy,fy,fy,fy],
                     'COHORT_SEMESTER':['FALL','FALL','FALL','FALL','FALL','FALL','FALL','FALL','FALL'],
                     'RACE':['WHITE','ASIAN','HISP','BLACK','TWO_PLUS','AMER_IND_AK','NAT_HAW_PI','NRA','UNKNOWN'],
                     'COHORT_SIZE':[white, asian, hispanic, black, two_plus, native_a, pacific_i, nra, unknown],
                     'TRAN_wi3':[white_t, asian_t, hispanic_t, black_t, two_plus_t, native_a_t, pacific_i_t, nra_t, unknown_t],
                     'TR':[TR_white, TR_asian, TR_hispanic, TR_black, TR_two_plus, TR_native_a, TR_pacific_i, TR_nra, TR_unknown]}, 
                     index = [1,2,3,4,5,6,7,8,9])
              ##df: first generation
              TR_fg = 0
              if fg_cs > 0:
                     TR_fg = fg_t/fg_cs*100 
              sdf_fg = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['FALL'],
                     'FIRST_GEN_MN':['Yes'],
                     'COHORT_SIZE':[fg_cs],
                     'TRAN_wi3':[fg_t],
                     'TR':[TR_fg]}, index=[1])
              ##df: overall
              TR = transferred_wi3_main/cohort_size_main*100
              sdf = pd.DataFrame(
                     {'COHORT_FY':fy,
                     'COHORT_SEMESTER':'FALL',
                     'COHORT_SIZE':cohort_size_main,
                     'TRAN_wi3':transferred_wi3_main,
                     'TRAN_at':transferred_at_main,
                     'TR':TR}, index=[1])
              ##apppend all new dfs onto the main ones
              trdf_gender = pd.concat([trdf_gender, sdf_gender])
              trdf_pell = pd.concat([trdf_pell, sdf_pell])
              trdf_race = pd.concat([trdf_race,sdf_race])
              trdf_FG = pd.concat([trdf_FG,sdf_fg])
              trdf = pd.concat([trdf,sdf])
       return trdf, trdf_gender, trdf_pell, trdf_FG, trdf_race, demo_transfers

In [6]:
def springTransferTally(df1,df2,df3,df4):
       trdf = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','COHORT_SIZE','TRAN_wi3','TRAN_at','TR'])
       trdf_race = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','RACE','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_FG = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','FIRST_GEN_MN','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_pell = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','PELL_ELG','COHORT_SIZE','TRAN_wi3','TR'])
       trdf_gender = pd.DataFrame(columns=['COHORT_FY','COHORT_SEMESTER','GENDER','COHORT_SIZE','TRAN_wi3','TR'])
       persDTL_2 = df1.copy()
       #create and fill new columns
       persDTL_2['transferred_wi3_spring'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED']
       persDTL_2['transferred_at'] = persDTL_2['FIRST_FALL_TRANSFERRED'] + persDTL_2['FIRST_SPRING_TRANSFERRED'] + persDTL_2['SECOND_FALL_TRANSFERRED'] + persDTL_2['SECOND_SPRING_TRANSFERRED'] + persDTL_2['THIRD_FALL_TRANSFERRED'] + persDTL_2['THIRD_SPRING_TRANSFERRED'] + persDTL_2['FOURTH_FALL_TRANSFERRED'] + persDTL_2['FOURTH_SPRING_TRANSFERRED'] + persDTL_2['FIFTH_FALL_TRANSFERRED'] + persDTL_2['FIFTH_SPRING_TRANSFERRED'] + persDTL_2['SIXTH_FALL_TRANSFERRED'] + persDTL_2['SIXTH_SPRING_TRANSFERRED'] + persDTL_2['SEVENTH_FALL_TRANSFERRED'] + persDTL_2['SEVENTH_SPRING_TRANSFERRED'] + persDTL_2['EIGTH_FALL_TRANSFERRED'] + persDTL_2['EIGTH_SPRING_TRANSFERRED']
       persDTL_2['grad_at'] = persDTL_2['FIRST_FALL_GRADUATED'] + persDTL_2['FIRST_SPRING_GRADUATED'] + persDTL_2['SECOND_FALL_GRADUATED'] + persDTL_2['SECOND_SPRING_GRADUATED'] + persDTL_2['THIRD_FALL_GRADUATED'] + persDTL_2['THIRD_SPRING_GRADUATED'] + persDTL_2['FOURTH_FALL_GRADUATED'] + persDTL_2['FOURTH_SPRING_GRADUATED'] + persDTL_2['FIFTH_FALL_GRADUATED'] + persDTL_2['FIFTH_SPRING_GRADUATED'] + persDTL_2['SIXTH_FALL_GRADUATED'] + persDTL_2['SIXTH_SPRING_GRADUATED'] + persDTL_2['SEVENTH_FALL_GRADUATED'] + persDTL_2['SEVENTH_SPRING_GRADUATED'] + persDTL_2['EIGTH_FALL_GRADUATED'] + persDTL_2['EIGTH_SPRING_GRADUATED']
       persDTL_2['retained'] = persDTL_2['FIRST_FALL_RETAINED'] + persDTL_2['FIRST_SPRING_RETAINED'] + persDTL_2['SECOND_FALL_RETAINED'] + persDTL_2['SECOND_SPRING_RETAINED'] + persDTL_2['THIRD_FALL_RETAINED'] + persDTL_2['THIRD_SPRING_RETAINED'] + persDTL_2['FOURTH_FALL_RETAINED'] + persDTL_2['FOURTH_SPRING_RETAINED'] + persDTL_2['FIFTH_FALL_RETAINED'] + persDTL_2['FIFTH_SPRING_RETAINED'] + persDTL_2['SIXTH_FALL_RETAINED'] + persDTL_2['SIXTH_SPRING_RETAINED'] + persDTL_2['SEVENTH_FALL_RETAINED'] + persDTL_2['SEVENTH_SPRING_RETAINED'] + persDTL_2['EIGTH_FALL_RETAINED'] + persDTL_2['EIGTH_SPRING_RETAINED']
       #get rid of the lengthy semester columns
       persDTL_3 = persDTL_2[['TECH_ID','FIRST_GENERATION_STATUS_MN','FIRST_GENERATION_STATUS_FED','PELL_ELIGIBLE','GENDER','FY','Semester','transferred_wi3_spring','transferred_at','grad_at','retained']]
       #merge df2 through df5
       last_admstat = pd.merge(df2, df3, how='left',on=['TECH_ID','YRTR'])
       last_admstat_race = pd.merge(last_admstat, df4,how='left',on='TECH_ID')
       #merge with the transfer df
       demo_transfers = pd.merge(persDTL_3, last_admstat_race, how='left',on='TECH_ID')
       fys = demo_transfers['FY'].unique()
       for fy in fys:
              df_main = demo_transfers.loc[(demo_transfers['FY']==fy),:]
              ##df_out1 variables
              cohort_size_main, transferred_wi3_main, transferred_at_main = 0,0,0
              ##df_out2 variables
              two_plus,native_a, pacific_i, hispanic, asian, white, black, nra, unknown = 0,0,0,0,0,0,0,0,0
              two_plus_t, native_a_t, pacific_i_t, hispanic_t, asian_t, white_t, black_t, nra_t, unknown_t = 0,0,0,0,0,0,0,0,0
              TR_white,TR_asian,TR_black,TR_hispanic,TR_two_plus,TR_native_a,TR_pacific_i,TR_nra,TR_unknown = 0,0,0,0,0,0,0,0,0
              ##df_out3 variables
              fg_cs, fg_t = 0,0
              ##df_out4 variables
              pell_elg, pell_elg_t = 0,0
              ##df_out5 variables
              male, female, male_t, female_t = 0,0,0,0
              df_main.reset_index()
              for i in df_main.index:
                     cohort_size_main = cohort_size_main + 1
                     at = df_main.loc[i,'transferred_at']
                     wi3 = df_main.loc[i,'transferred_wi3_spring']
                     fg = df_main.loc[i,'FIRST_GENERATION_STATUS_MN']
                     race = df_main.loc[i,'IPEDS']
                     pell = df_main.loc[i,'PELL_ELIGIBLE']
                     gender = df_main.loc[i,'GENDER']
                     if race == 'WHITE':
                            white = white + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   white_t = white_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'HISP':
                            hispanic = hispanic + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   hispanic_t = hispanic_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'BLACK':
                            black = black + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   black_t = black_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'ASIAN':
                            asian = asian + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   asian_t = asian_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'TWO_PLUS':
                            two_plus = two_plus + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   two_plus_t = two_plus_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'AMER_IND_AK':
                            native_a = native_a + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   native_a_t = native_a_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NAT_HAW_PI':
                            pacific_i = pacific_i + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   pacific_i_t = pacific_i_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'NRA':
                            nra = nra + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   nra_t = nra_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     elif race == 'UNKNOWN':
                            unknown = unknown + 1
                            if wi3 > 0:
                                   transferred_wi3_main = transferred_wi3_main + 1
                                   unknown_t = unknown_t + 1
                                   if fg == 'Yes':
                                          fg_t = fg_t + 1
                                   if pell == 'Yes':
                                          pell_elg_t = pell_elg_t + 1
                                   if gender == 'Female':
                                          female_t = female_t + 1
                                   if gender == 'Male':
                                          male_t = male_t + 1
                            if at > 0:
                                   transferred_at_main = transferred_at_main + 1
                            if fg == 'Yes':
                                   fg_cs = fg_cs + 1       
                            if pell == 'Yes':
                                   pell_elg = pell_elg + 1
                            if gender == 'Female':
                                   female = female + 1
                            if gender == 'Male':
                                   male = male + 1
                     else:
                            continue
              ##Race transfer rate; this doesn't seem necessary, but some of these groupings are 0 size, which will break that function should a 0 end up in the denominator 
              if white > 0:
                     TR_white = white_t/white*100   
              if asian > 0:
                     TR_asian = asian_t/asian*100   
              if black > 0:
                     TR_black = black_t/black*100   
              if hispanic > 0:
                     TR_hispanic = hispanic_t/hispanic*100   
              if two_plus > 0:
                     TR_two_plus = two_plus_t/two_plus*100 
              if native_a > 0:  
                     TR_native_a = native_a_t/native_a*100   
              if pacific_i > 0:
                     TR_pacific_i = pacific_i_t/pacific_i*100   
              if nra > 0:
                     TR_nra = nra_t/nra*100   
              if unknown > 0:
                     TR_unknown = unknown_t/unknown*100  
              #df: gender
              TR_fem = 0
              if female > 0:
                     TR_fem = female_t/female*100
              TR_male = 0
              if male > 0:
                     TR_male = male_t/male*100
              sdf_gender = pd.DataFrame(
                     {'COHORT_FY':[fy,fy],
                     'COHORT_SEMESTER':['SPRING','SPRING'],
                     'GENDER':['Female','Male'],
                     'COHORT_SIZE':[female, male],
                     'TRAN_wi3':[female_t, male_t],
                     'TR':[TR_fem, TR_male]}, index=[1,2])
              #df: pell
              TR_pell = 0
              if pell_elg > 0:
                     TR_pell = pell_elg_t/pell_elg*100
              sdf_pell = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['SPRING'],
                     'PELL_ELG':['Yes'],
                     'COHORT_SIZE':[pell_elg],
                     'TRAN_wi3':[pell_elg_t],
                     'TR':[TR_pell]}, index=[1])
              #df: race
              sdf_race = pd.DataFrame(
                     {'COHORT_FY':[fy,fy,fy,fy,fy,fy,fy,fy,fy],
                     'COHORT_SEMESTER':['SPRING','SPRING','SPRING','SPRING','SPRING','SPRING','SPRING','SPRING','SPRING'],
                     'RACE':['WHITE','ASIAN','HISP','BLACK','TWO_PLUS','AMER_IND_AK','NAT_HAW_PI','NRA','UNKNOWN'],
                     'COHORT_SIZE':[white, asian, hispanic, black, two_plus, native_a, pacific_i, nra, unknown],
                     'TRAN_wi3':[white_t, asian_t, hispanic_t, black_t, two_plus_t, native_a_t, pacific_i_t, nra_t, unknown_t],
                     'TR':[TR_white, TR_asian, TR_hispanic, TR_black, TR_two_plus, TR_native_a, TR_pacific_i, TR_nra, TR_unknown]}, 
                     index = [1,2,3,4,5,6,7,8,9])
              ##df: first generation
              TR_fg = 0
              if fg_cs > 0:
                     TR_fg = fg_t/fg_cs*100 
              sdf_fg = pd.DataFrame(
                     {'COHORT_FY':[fy],
                     'COHORT_SEMESTER':['SPRING'],
                     'FIRST_GEN_MN':['Yes'],
                     'COHORT_SIZE':[fg_cs],
                     'TRAN_wi3':[fg_t],
                     'TR':[TR_fg]}, index=[1])
              ##df: overall
              TR = transferred_wi3_main/cohort_size_main*100
              sdf = pd.DataFrame(
                     {'COHORT_FY':fy,
                     'COHORT_SEMESTER':'SPRING',
                     'COHORT_SIZE':cohort_size_main,
                     'TRAN_wi3':transferred_wi3_main,
                     'TRAN_at':transferred_at_main,
                     'TR':TR}, index=[1])
              ##apppend all new dfs onto the main ones
              trdf_gender = pd.concat([trdf_gender, sdf_gender])
              trdf_pell = pd.concat([trdf_pell, sdf_pell])
              trdf_race = pd.concat([trdf_race,sdf_race])
              trdf_FG = pd.concat([trdf_FG,sdf_fg])
              trdf = pd.concat([trdf,sdf])
       return trdf, trdf_gender, trdf_pell, trdf_FG, trdf_race, demo_transfers
    

In [8]:
#d1,d2,d3,d4,d5,d6 = fallTransferTally(persDTL_fall, last, admstat_10, race_10)
d7,d8,d9,d10,d11,d12 = springTransferTally(persDTL_spring, last, admstat_10, race_10)
#Colleges_allTime, Colleges_lastCompleteCohort, Colleges_last5CompleteCohorts, allTransfers, publicVSprivate_type_state = collegeTransfers(persDTL2_10, last, race_10, transferRecords_10)

In [40]:
##to_csv for TransferTally functions
d13 = pd.concat([d1,d7])
d14 = pd.concat([d2,d8])
d15 = pd.concat([d3,d9])
d16 = pd.concat([d4,d10])
d17 = pd.concat([d5,d11])
d18 = pd.concat([d6,d12])
d13.to_csv('d13.csv', index=False)
d14.to_csv('d14.csv', index=False)
d15.to_csv('d15.csv', index=False)
d16.to_csv('d16.csv', index=False)
d17.to_csv('d17.csv', index=False)
d18.to_csv('d18.csv', index=False)

In [ ]:
##to_csv for collegeTransfers function
Colleges_allTime.to_csv('Colleges_allTime.csv', index=False)
Colleges_lastCompleteCohort.to_csv('Colleges_lastCompleteCohort.csv', index=False)
Colleges_last5CompleteCohorts.to_csv('Colleges_last5CompleteCohorts.csv', index=False)
allTransfers.to_csv('allTransfers.csv', index=False)
publicVSprivate_type_state.to_csv('publicVSprivate_type_state.csv', index=False)

In [39]:
#d13.sort_values(by=['FY','Semester','transferred_wi3','TECH_ID'])
d13.sort_values(by=['COHORT_FY','COHORT_SEMESTER'])
#d11.sort_values(by=['COHORT_FY','COHORT_SEMESTER'])


,COHORT_FY,COHORT_SEMESTER,COHORT_SIZE,TRAN_wi3,TRAN_at,TR
1,2013,FALL,748,109,144,14.572193
1,2013,SPRING,233,60,71,25.751073
1,2014,FALL,669,121,159,18.086697
1,2014,SPRING,243,45,57,18.518519
1,2015,FALL,616,108,142,17.532468
1,2015,SPRING,161,38,45,23.602484
1,2016,FALL,566,97,132,17.137809
1,2016,SPRING,161,23,32,14.285714
1,2017,FALL,620,79,114,12.741935
1,2017,SPRING,142,25,28,17.605634
